# Table of Contents
* &nbsp;
	* &nbsp;
		* [Experimentation -- Noise cancellation ](#Experimentation----Noise-cancellation)
			* [Implementation of the noise canceller](#Implementation-of-the-noise-canceller)
			* [First experiment ](#First-experiment)
			* [Second experiment](#Second-experiment)
			* [Corrupted speech](#Corrupted-speech)


In [15]:
%run nbinit.ipy

... Configuring matplotlib formats
... Configuring matplotlib with inline figures
... Importing numpy as np, scipy as sp, pyplot as plt, scipy.stats as stats
   ... scipy.signal as sig
... Importing widgets, display, HTML, Image, Javascript
... Some LaTeX definitions


... Defining figures captions 


... Loading customized Javascript for interactive solutions (show/hide)


## Experimentation -- Noise cancellation 


<img src="noisecancellation.jpg"  width="60%" height="60%">


Let $s$ be the signal we want to estimate from the mixture $x=s+b$, and from the noise reference $u$.  You will apply a noise cancellation procedure, where the filter will be identified in an adaptive way, using a LMS algorithm. 


### Implementation of the noise canceller

 
You will implement a function following the call syntax


In [ ]:
def noisecancel(ref,signal,h_ini,mu,normalized=False):
    """
    Noise cancellation algorithm
    Inputs:
    =======
    ref: array
        noise reference
    signal:  array
        signal path (signal mixture s +b, where b is correlated with ref
    h_ini: array
        initial impulse response
    mu: adaptation step
    Outputs:
    ========
    h: array
        identified impulse response 
    s: array
        signal identified by noise cancellation
    b_est: array
        noise on the signal path estimated from ref
    """
    #
    # DO IT YOURSELF!
    #
    #return (h,s,b_est)

In [8]:
from IPython.display import HTML, display, Javascript
display(HTML('<div class="onoffswitch">\n<input type="checkbox" name="onoffswitch" class="onoffswitch-checkbox"\nonclick="document.getElementById('+"'myCheck23'"+').checked ? \nshow_input(2) : hide_input(2)" \nid="myCheck23"  checked>\n<label class="onoffswitch-label" for="myCheck23"> \n<span class="onoffswitch-inner"></span> \n<span class="onoffswitch-switch"></span>\n</label>\n</div>'))

In [ ]:
def lms(d,u,w,mu):
    """ 
    Implements a single iteration of the stochastic gradient (LMS)\n
    :math:`w(n+1)=w(n)+\\mu u(n)\\left(d(n)-w(n)^T u(n)\\right)̀`
    
    Input:
    ======
        d : desired sequence at time n 
        u : input of length p
        w : wiener filter to update 
        mu : adaptation step
    
    Returns:
    =======
        w : upated filter
        err : d-dest
        dest : prediction = :math:`u(n)^T w` 
    """
    u=squeeze(u) #Remove single-dimensional entries from the shape of an array.
    w=squeeze(w)
    dest=u.dot(w)
    err=d-dest
    w=w+mu*u*err
    return (w,err,dest)

In [ ]:
def noisecancel(ref,signal,h_ini,mu,normalized=False):
    """
    Noise cancellation algorithm
    Inputs:
    =======
    ref: array
        noise reference
    signal:  array
        signal path (signal mixture s +b, where b is correlated with ref
    h_ini: array
        initial impulse response
    mu: adaptation step
    
    Outputs:
    ========
    h: array
        identified impulse response 
    s: array
        signal identified by noise cancellation
    b_est: array
        noise on the signal path estimated from ref
    """


    ref=np.squeeze(ref) 
    signal=np.squeeze(signal) 
    h_ini=np.squeeze(h_ini)

    N=np.size(signal)
    L=np.size(h_ini)
    h=h_ini
    s=np.zeros(N)
    b_est=np.zeros(N)
    
    for t in np.arange(L,N):
         u=ref[t:t-L:-1]
         if normalized:
            mun=mu/(dot(u,u)+1e-10)   
         else:
            mun=mu
         (h, s[t],b_est[t]) = lms(signal[t], u, h, mun)

    return (h,s,b_est)

You may first test your implementation using the following test script.

In [ ]:
# Test: Problem Simulation
N=1000
u=np.random.randn(N)
v=sig.lfilter(ones(10),1,u)
s=sin(2*pi*0.03*np.arange(N))
x=s+v
(h,s,s_est)=noisecancel(u,x,zeros(12),0.2,normalized=True)
plt.figure()
plt.plot(x)
plt.title("Observations")
plt.figure()
plt.plot(v)
plt.title("Noise reference")
plt.figure()
plt.plot(s)
plt.title("Cleaned signal")
print(h)

### First experiment 

For your experiments, you have to consider the tests signals included in the files 
`sb1.npz` and `sb2.npz`. The first file contains two noise references, one is stationary, the second non stationary. You will consider these two references successively. You will take values between  0.01 and 1 for the adaptation step (or you may use the normalized version). 


We must first load the data:

    f=numpy.load('sb1.npz')

In [ ]:
f=np.load('sb1.npz')
# f is a dictionary
# its keys are given by
print("keys: ", f.keys())
# Then the contents are affected to local variables
obs=f['obs']
ref1=f['ref1']
ref2=f['ref2']

Plot the different signals

In [9]:
from IPython.display import HTML, display, Javascript
display(HTML('<div class="onoffswitch">\n<input type="checkbox" name="onoffswitch" class="onoffswitch-checkbox"\nonclick="document.getElementById('+"'myCheck9'"+').checked ? \nshow_input(1) : hide_input(1)" \nid="myCheck9"  checked>\n<label class="onoffswitch-label" for="myCheck9"> \n<span class="onoffswitch-inner"></span> \n<span class="onoffswitch-switch"></span>\n</label>\n</div>'))

In [ ]:
import matplotlib.gridspec as gridspec
G = gridspec.GridSpec(2, 2)

fig=plt.figure(figsize=(10,4))
ax1 = plt.subplot(G[0:, 0])
ax1.plot(obs)
title("Observation")
ax2 = subplot(G[0, 1])
ax2.plot(ref1)
plt.title("Noise reference 1")

ax3 = subplot(G[1, 1])
ax3.plot(ref2)
plt.title("Noise reference 2")
fig.tight_layout()   #avoid covering of titles and labels
figcaption("Noise cancellation experiment -- Available signals",label="fig:availablesignals1")

Test the noise cancellation algorithm, using the two references.

In [10]:
from IPython.display import HTML, display, Javascript
display(HTML('<div class="onoffswitch">\n<input type="checkbox" name="onoffswitch" class="onoffswitch-checkbox"\nonclick="document.getElementById('+"'myCheck10'"+').checked ? \nshow_input(1) : hide_input(1)" \nid="myCheck10"  checked>\n<label class="onoffswitch-label" for="myCheck10"> \n<span class="onoffswitch-inner"></span> \n<span class="onoffswitch-switch"></span>\n</label>\n</div>'))

In [ ]:
# With the non stationary reference
(h,s,sest)=noisecancel(ref2,obs,zeros(10),0.8,normalized=True)
plt.plot(s)
plt.title("Extracted signal")
figcaption("Extracted signal",label="fig:extractedsignal1")
plt.figure()
plt.plot(sest)
plt.title("Extracted noise")
figcaption("Extracted noise",label="fig:extractednoise1")


In [11]:
from IPython.display import HTML, display, Javascript
display(HTML('<div class="onoffswitch">\n<input type="checkbox" name="onoffswitch" class="onoffswitch-checkbox"\nonclick="document.getElementById('+"'myCheck11'"+').checked ? \nshow_input(1) : hide_input(1)" \nid="myCheck11"  checked>\n<label class="onoffswitch-label" for="myCheck11"> \n<span class="onoffswitch-inner"></span> \n<span class="onoffswitch-switch"></span>\n</label>\n</div>'))

In [ ]:
# With the stationary reference
(h,s,sest)=noisecancel(ref1,obs,zeros(10),0.8,normalized=True)
plt.plot(s)
plt.title("Extracted signal")
figcaption("Extracted signal",label="fig:extractedsignal2")
plt.figure()
plt.plot(sest)
plt.title("Extracted noise")
figcaption("Extracted noise",label="fig:extractednoise2")


### Second experiment

For the second signal in  `sb2.npz`, we have a much more important non stationarity and a lower signal-to-noise ratio. It might be useful to consider the problem in two phases, so as to obtain a frst estimate of the impulse response, which will be used as an initial condition in the second phase. 

In [ ]:
f=np.load('sb2.npz')
# f is a dictionary
# its keys are given by
print("keys: ", f.keys())
#  Then the contents are affected to local variables
Text=f['Text']
x=f['x']
y=f['y']

Print the Text variable, and display the different signals. 

In [12]:
from IPython.display import HTML, display, Javascript
display(HTML('<div class="onoffswitch">\n<input type="checkbox" name="onoffswitch" class="onoffswitch-checkbox"\nonclick="document.getElementById('+"'myCheck12'"+').checked ? \nshow_input(1) : hide_input(1)" \nid="myCheck12"  checked>\n<label class="onoffswitch-label" for="myCheck12"> \n<span class="onoffswitch-inner"></span> \n<span class="onoffswitch-switch"></span>\n</label>\n</div>'))

In [ ]:
print(Text) #
plot(x)
title("x: Noise reference")
figure()
plot(y)
title("y = s + b : available data")

Call the noise canceller in order to extract signal.

In [13]:
from IPython.display import HTML, display, Javascript
display(HTML('<div class="onoffswitch">\n<input type="checkbox" name="onoffswitch" class="onoffswitch-checkbox"\nonclick="document.getElementById('+"'myCheck13'"+').checked ? \nshow_input(1) : hide_input(1)" \nid="myCheck13"  checked>\n<label class="onoffswitch-label" for="myCheck13"> \n<span class="onoffswitch-inner"></span> \n<span class="onoffswitch-switch"></span>\n</label>\n</div>'))

In [ ]:
(h,s,sest)=noisecancel(x,y,zeros(10),0.8,normalized=True)
(h,s,sest)=noisecancel(x,y,h,0.1,normalized=True)
# We assume that the filter is stationary and filter again 
# with this filter as initial condition 
(h,s,sest)=noisecancel(x,y,h,0.0001,normalized=True)
plt.plot(s)
plt.title("Extracted signal")
figcaption("Extracted signal",label="fig:extractedsignal3")
plt.figure()
plt.plot(sest)
plt.title("Extracted noise")
figcaption("Extracted noise",label="fig:extractednoise3")

print("Identified filter: ",h)

### Corrupted speech

We end with a speech sound, corrupted by a cricket chirping 

1. We load data by
   
        f=numpy.load('parole_bruitee.npz')  
        g=numpy.load('decticelle.npz')
        
   then the contents are affected to local variables

        d=f['d']
        u=g['u']
2.   Perform the noise cancellation. Then you will be able to listen the result 
- using the Audio object if you use the IPython notebook
```
from IPython.display import Audio
Audio(data=name_of_array,rate=value_of-rate) #eg rate=8820
```
- using the `sound` function which has been thrown together by your servant 
```
from syssound import *
sound(name_of_array)
```

In [ ]:
f=np.load('parole_bruitee.npz')
print(f.keys())
g=np.load('decticelle.npz')
g.keys()

Call the noisecanceller, display the results and listen the different signals... 

In [14]:
from IPython.display import HTML, display, Javascript
display(HTML('<div class="onoffswitch">\n<input type="checkbox" name="onoffswitch" class="onoffswitch-checkbox"\nonclick="document.getElementById('+"'myCheck22'"+').checked ? \nshow_input(4) : hide_input(4)" \nid="myCheck22"  checked>\n<label class="onoffswitch-label" for="myCheck22"> \n<span class="onoffswitch-inner"></span> \n<span class="onoffswitch-switch"></span>\n</label>\n</div>'))

In [ ]:
d=f['d']
u=g['u']
(h,s,sest)=noisecancel(u,d,zeros(5),0.1,normalized=True)
plt.plot(d)
plt.title("Corrupted Signal")
figcaption("Corrupted Signal", label="fig:corruptedsignal")
plt.figure()
plt.plot(s)
plt.title("Recovered Speech?")
figcaption("Recovered speech", label="fig:recoveredspeech")

In [ ]:
from IPython.display import Audio

In [ ]:
# Corrupted signal
Audio(data=d,rate=8820)

In [ ]:
# Restored one signal
Audio(data=s,rate=8820)